<a href="https://colab.research.google.com/github/clyde2020/git-github/blob/master/Q5_with_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
!pip install keras-tuner -q
from keras_tuner import RandomSearch

# How about this random ass comment
# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

# DO NOT CHANGE THIS.
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

# COMPLETE THE CODE IN THIS FUNCTION
df = pd.read_csv('/content/Weekly_U.S.Diesel_Retail_Prices.csv',
                     infer_datetime_format=True, index_col='Week of', header=0)

N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

# Normalizes the data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

# Splits the data into training and validation sets.
SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

# DO NOT CHANGE THIS CODE
tf.keras.backend.clear_session()
tf.random.set_seed(42)

BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS

# DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
# on the server.
# Number of past time steps based on which future observations should be
# predicted
N_PAST = 10  # DO NOT CHANGE THIS

# Number of future time steps which are to be predicted.
N_FUTURE = 10  # DO NOT CHANGE THIS

# By how many positions the window slides to create a new window
# of observations.
SHIFT = 1  # DO NOT CHANGE THIS

     |████████████████████████████████| 98 kB 2.8 MB/s 


In [ ]:
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                              n_past=N_PAST, n_future=N_FUTURE,
                              shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                              n_past=N_PAST, n_future=N_FUTURE,
                              shift=SHIFT)

In [ ]:
def build_model(hp):
  model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=hp.Choice('filters', values=[40, 44, 48]), kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hp.Choice('units1', values=[40, 44, 48, 52, 56, 60, 64]), return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hp.Choice('units2', values=[48, 52, 56]), return_sequences=True)),
  tf.keras.layers.Dense(units=hp.Choice('units3', values=[8, 10, 12]), activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  model.compile(loss=tf.keras.losses.Huber(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=["mae"])
  
  return model

     |████████████████████████████████| 98 kB 3.7 MB/s 


In [ ]:
tuner = RandomSearch(
    build_model,
    objective="val_mae",
    max_trials=50,
    executions_per_trial=2,
    overwrite=True,
    directory="keras_tuner_directory",
    project_name="Q5_experiment",
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
filters (Choice)
{'default': 40, 'conditions': [], 'values': [40, 44, 48], 'ordered': True}
units1 (Choice)
{'default': 40, 'conditions': [], 'values': [40, 44, 48, 52, 56, 60, 64], 'ordered': True}
units2 (Choice)
{'default': 48, 'conditions': [], 'values': [48, 52, 56], 'ordered': True}
units3 (Choice)
{'default': 8, 'conditions': [], 'values': [8, 10, 12], 'ordered': True}


In [ ]:
tuner.search(train_set, epochs=40, validation_data=valid_set)
tuner.results_summary()

Trial 17 Complete [00h 01m 51s]
val_mae: 0.027965671382844448

Best val_mae So Far: 0.027735497802495956
Total elapsed time: 00h 30m 21s

Search: Running Trial #18

Hyperparameter    |Value             |Best Value So Far 
filters           |48                |44                
units1            |44                |56                
units2            |56                |48                
units3            |12                |8                 



KeyboardInterrupt: ignored

In [ ]:
tuner.results_summary()

Results summary
Results in keras_tuner_directory/Q5_experiment
Showing 10 best trials
Objective(name='val_mae', direction='min')
Trial summary
Hyperparameters:
filters: 48
units1: 40
units2: 48
units3: 8
Score: 0.02688445057719946
Trial summary
Hyperparameters:
filters: 40
units1: 64
units2: 48
units3: 12
Score: 0.027346661314368248
Trial summary
Hyperparameters:
filters: 40
units1: 64
units2: 56
units3: 8
Score: 0.027736286632716656
Trial summary
Hyperparameters:
filters: 40
units1: 56
units2: 64
units3: 16
Score: 0.027869717217981815
Trial summary
Hyperparameters:
filters: 48
units1: 64
units2: 24
units3: 16
Score: 0.028256026096642017
Trial summary
Hyperparameters:
filters: 32
units1: 24
units2: 24
units3: 16
Score: 0.02844720333814621
Trial summary
Hyperparameters:
filters: 24
units1: 48
units2: 24
units3: 12
Score: 0.02862699795514345
Trial summary
Hyperparameters:
filters: 24
units1: 56
units2: 56
units3: 16
Score: 0.028665159828960896
Trial summary
Hyperparameters:
filters: 40
u